In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import PIL
import zipfile
import os

In [2]:
TRAIN_DATA_DIR = './data/train/'
VALIDATION_DATA_DIR = './data/val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Загрузка данных

Цветные изображения обычно имеют 3 канала: красный, зеленый и синий, каждый из которых имеет значение интенсивности в диапазоне от 0 до 255. Чтобы его нормализовать (т.е. привести значение от 0 до 1) можно изменить масштаб изображения, разделив каждый пиксель на 255. Либо использовать значение по умолчанию ` preprocess_input` в Keras, которая сама выполняет предварительную обработку.

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

- Обучение по одному изображению за раз может быть довольно неэффективным, поэтому следует объединить их в батчи.
- Чтобы добавить больше случайности в процесс обучения будет использоваться перемешивание в каждом батче shuffle=True
- Чтобы обеспечить воспроизводимость при нескольких запусках одной и той же программы, генератору случайных чисел задается seed.

In [5]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


### Создает кастомную модель глубокого обучения на основе предварительно обученной модели MobileNet.

In [6]:
# Загружает базовую модель MobileNet, без верхнего слоя (include_top=False) и с указанием размера входных изображений (IMG_WIDTH, IMG_HEIGHT, 3)
# Устанавливает все слои базовой модели как необучаемые (trainable=False), чтобы использовать их в качестве экстрактора признаков.
# Создает новый входной слой с тем же размером, что и входные изображения.
# Применяет базовую модель MobileNet к новому входному слою, получая некоторое количество карт признаков.
# Добавляет слой GlobalAveragePooling2D, чтобы сжать карты признаков в один вектор.
# Добавляет полносвязный слой с 64 нейронами и функцией активации ReLU.
# Добавляет слой Dropout с коэффициентом 0.2 для регуляризации (для больших датасетов можно увеличить до 0.5)
# Добавляет выходной полносвязный слой с NUM_CLASSES нейронами и функцией активации softmax для классификации
# Возвращает итоговую модель, которая принимает на вход изображения размера (IMG_WIDTH, IMG_HEIGHT, 3) и выдает предсказания для NUM_CLASSES классов.

def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.2)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [7]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
8/8 [==============================] - 30s 2s/step - loss: 0.5989 - acc: 0.7360 - val_loss: 0.1808 - val_acc: 0.9500
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 0.1843 - acc: 0.9240 - val_loss: 0.1221 - val_acc: 0.9580
Epoch 3/10
8/8 [==============================] - 12s 2s/step - loss: 0.1208 - acc: 0.9520 - val_loss: 0.0837 - val_acc: 0.9700
Epoch 4/10
8/8 [==============================] - 12s 2s/step - loss: 0.0892 - acc: 0.9640 - val_loss: 0.0792 - val_acc: 0.9640
Epoch 5/10
8/8 [==============================] - 12s 2s/step - loss: 0.0797 - acc: 0.9620 - val_loss: 0.0721 - val_acc: 0.9660
Epoch 6/10
8/8 [==============================] - 12s 2s/step - loss: 0.0498 - acc: 0.9880 - val_loss: 0.0667 - val_acc: 0.9680
Epoch 7/10
8/8 [==============================] - 12s 2s/step - loss: 0.0592 - acc: 0.9680 - val_loss: 0.0646 - val_acc: 0.9780
Epoch 8/10
8/8 [==============================] - 12s 2s/step - loss: 0.0490 - acc: 0.9800 - val_loss: 0

In [8]:
model.save('model.h5')

# Предикт модели

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [12]:
img_path = './sample-images/dog.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[4.2966454e-05 9.9995708e-01]]
{'cat': 0, 'dog': 1}
